In [ ]:
import random
from enum import Enum

In [ ]:
# --- Constants and Enumerations Definitions ---

# Fixed System Parameters (Example values)
L = 200        # Road Length (cells)
V_MAX = 5      # Maximum Speed (cells/time step)
T_GREEN = 40   # Green Light Period (time steps)
T_CYCLE = 2 * T_GREEN # Full Cycle (assuming symmetry)
INJECTION_RATE = 0.1 # Probability of injection 'a'

# Base Parameters (Example values)
P_B = 0.1      # Random Braking Probability (NaSch)
P_CHG = 0.8    # Lane Change Probability
P_RED = 0.1    # Red Light Violation Probability
P_SKID = 0.05  # Braking Failure Probability (Rear-end collision)

class Road(Enum):
    """Represents the two perpendicular roads of the intersection."""
    R1 = 1 # Vertical Axis (North-South)
    R2 = 2 # Horizontal Axis (West-East)
    
class Lane(Enum):
    """Represents the two lanes of a road."""
    LEFT = 0
    RIGHT = 1

class TrafficLightState(Enum):
    """Traffic light state for a road (R1 or R2)."""
    RED = 0
    GREEN = 1

class Weather(Enum):
    """Weather condition for the simulation."""
    NORMAL = 0
    RAINY = 1 # "Rainy day" state (global modifier)

# ----------------------------------------------------
# Vehicle Class
# ----------------------------------------------------

class Vehicle:
    """Represents a single vehicle in the model."""
    def __init__(self, vehicle_id, road: Road, lane: Lane, vmax, p_red, p_skid):
        self.id = vehicle_id
        self.road = road # R1 or R2
        self.lane = lane # LEFT or RIGHT
        self.position = 0
        self.velocity = 0
        self.v_max = vmax
        
        # Individual parameters (can be modified by weather)
        self.p_red = p_red
        self.p_skid = p_skid
        
        # Collision status
        self.collided = False

# ----------------------------------------------------
# Intersection Model Class
# ----------------------------------------------------

class IntersectionModel:
    """
    Main simulation managing roads, vehicles, traffic light, and accidents.
    It's an extension of the Nagel-Schreckenberg model.
    """
    def __init__(self, length, vmax, t_green, injection_rate, p_b, p_chg, p_red, p_skid):
        
        # Simulation Parameters
        self.L = length
        self.V_MAX_BASE = vmax
        self.T_GREEN = t_green
        self.INJECTION_RATE = injection_rate
        self.P_B_BASE = p_b
        self.P_CHG_BASE = p_chg
        
        # Simulation objects collections
        self.grid = {
            r: {l: [None] * self.L for l in Lane}
            for r in Road
        }
        self.vehicles = []
        self.next_vehicle_id = 0
        self.time_step = 0
        
        # Traffic Light
        self.traffic_light = {Road.R1: TrafficLightState.GREEN, Road.R2: TrafficLightState.RED}
        
        # Base Risk Parameters
        self.P_RED_BASE = p_red
        self.P_SKID_BASE = p_skid
        
        # Output Variables (Metrics to Measure)
        self.N_lateral = 0  # Total count of lateral collisions (red light violation)
        self.N_rear_end = 0  # Total count of rear-end collisions (braking failure)
        self.N_vehicles = 0 # Total number of vehicles that have circulated
        self.throughput = 0 # Number of vehicles crossing the intersection per unit time
        
        # Environmental state and modifiers
        self.weather = Weather.NORMAL
        self.P_B = self.P_B_BASE
        self.P_CHG = self.P_CHG_BASE
        self.P_RED = self.P_RED_BASE
        self.P_SKID = self.P_SKID_BASE

    def set_weather(self, weather: Weather):
        """Applies the modifiers for the weather condition (rainy day)."""
        self.weather = weather
        if weather == Weather.RAINY:
            # Modifications due to rain
            self.P_RED = self.P_RED_BASE * 0.5  # P_red is reduced (more cautious)
            self.P_CHG = self.P_CHG_BASE * 0.5  # P_chg is reduced (discourages lane changes)
            self.P_SKID = self.P_SKID_BASE * 2.0 # P_skid increases significantly (braking failure/aquaplaning)
            self.P_B = self.P_B_BASE * 1.5      # P_b increases (longer braking distance/more cautious braking)
        else:
            # Revert to base values
            self.P_RED = self.P_RED_BASE
            self.P_CHG = self.P_CHG_BASE
            self.P_SKID = self.P_SKID_BASE
            self.P_B = self.P_B_BASE

        # Update risk parameters in all existing vehicles
        for vehicle in self.vehicles:
            vehicle.p_red = self.P_RED
            vehicle.p_skid = self.P_SKID
            
    def update_traffic_light(self):
        """Updates the traffic light state every T_GREEN time steps."""
        # Full cycle T = 2 * T_GREEN
        if (self.time_step % self.T_GREEN) == 0:
            # Switch lights
            if self.traffic_light[Road.R1] == TrafficLightState.GREEN:
                self.traffic_light[Road.R1] = TrafficLightState.RED
                self.traffic_light[Road.R2] = TrafficLightState.GREEN
            else:
                self.traffic_light[Road.R1] = TrafficLightState.GREEN
                self.traffic_light[Road.R2] = TrafficLightState.RED

    def inject_vehicle(self):
        """Tries to inject a new vehicle into the system."""
        # Implementation of vehicle injection with INJECTION_RATE 'a'
        # Should choose a road (R1/R2) and a lane (LEFT/RIGHT) if the starting cell (position 0) is free.
        pass # Injection logic (to be implemented)
        
    def find_front_gap(self, vehicle):
        """Calculates the distance (gap) to the vehicle ahead or to the intersection."""
        # The true gap (d_i) must consider the next vehicle and the intersection (L/2).
        return self.L # Initial simplicity (if no car ahead)

    def find_front_vehicle(self, vehicle):
        """Finds the vehicle directly ahead."""
        # Requires searching ahead in the same lane/road.
        return None

    def apply_nash_rules(self):
        """Applies the NaSch update rules to each vehicle in parallel."""
        new_vehicles_state = {} # Stores the temporary new position and velocity
        
        for vehicle in self.vehicles:
            if vehicle.collided:
                continue

            v_i = vehicle.velocity
            d_i = self.find_front_gap(vehicle) # distance to car ahead or to stop-line (intersection)
            
            # **1. Acceleration**
            v_i_prime = min(v_i + 1, vehicle.v_max)
            
            # **2. Deceleration by vehicle ahead (Rule R2)**
            v_i_double_prime = min(v_i_prime, d_i)
            
            # **Deceleration by Red Light**
            if self.traffic_light[vehicle.road] == TrafficLightState.RED:
                intersection_pos = self.L // 2
                g_i = intersection_pos - vehicle.position - 1 # Space to the cell before the intersection
                
                # Check for Red Light Violation (Lateral Collision Risk)
                if g_i <= 0 and random.random() < vehicle.p_red:
                    # **Lateral Collision: Red Light Violation**
                    # This happens if the car runs the red light, enters the intersection,
                    # AND a car from R2 (with green) also enters.
                    # A collision check in the intersection site is needed here.
                    self.N_lateral += 1 # Placeholder increment; detailed logic needed
                    vehicle.collided = True
                    # continue # Stop processing this vehicle for this step
                
                v_i_double_prime = min(v_i_double_prime, g_i) # Brake before the stop-line
            
            # **3. Randomization (Rule R3)**
            v_i_triple_prime = v_i_double_prime
            if v_i_triple_prime > 0 and random.random() < self.P_B: # P_B is modified by rain
                v_i_triple_prime -= 1 # Random braking
                
            # **4. Car Motion**
            new_v = v_i_triple_prime
            new_pos = vehicle.position + new_v
            
            # **5. Braking Failure Check (Rear-End Collision)**
            # Check if the intended movement (new_pos) exceeds the safety gap (position + d_i)
            # This check is complex and dependent on how d_i is calculated.
            # A simplified check for a potential rear-end collision:
            if new_v > d_i and random.random() < vehicle.p_skid:
                # Car was supposed to slow down to v <= d_i (Rule R2), but new_v > d_i implies Rule R2 failed
                # due to the P_skid probability. 
                self.N_rear_end += 1 # Rear-end collision
                vehicle.collided = True
                # The car stops (or moves to the cell behind the front car)
                new_pos = vehicle.position 
                new_v = 0
                
            # Store the new position/velocity
            new_vehicles_state[vehicle] = {'pos': new_pos, 'vel': new_v}

        # **6. Lane Changing (P_chg)**
        # This step should happen before final position update.
        # It involves: Safety and congestion checks in the adjacent lane, applied with probability P_CHG.
        # P_CHG is affected by rain. This is a complex step (to be implemented).
        
        # **7. Apply Final State and Update Grid**
        # Move vehicles to new_pos, handle vehicles leaving the system (throughput).
        
        self.time_step += 1
        
    def run_simulation(self, steps, rainy=False):
        """Runs the simulation for a number of steps."""
        self.set_weather(Weather.RAINY if rainy else Weather.NORMAL)
        
        for _ in range(steps):
            self.update_traffic_light()
            self.inject_vehicle()
            self.apply_nash_rules()
            # Missing: data collection/animation logic
            # Missing: handling of collided vehicles (removal/blocking)

    def get_metrics(self):
        """Returns the output metrics of the simulation."""
        return {
            'N_lateral': self.N_lateral,
            'N_rear_end': self.N_rear_end,
            'N_vehicles': self.N_vehicles,
            'Throughput': self.throughput,
            'Accident_Ratio_Lateral_to_RearEnd': self.N_lateral / (self.N_rear_end + 1e-6)
        }

In [ ]:
# ----------------------------------------------------
# Execution (Example of usage)
# ----------------------------------------------------

if __name__ == "__main__":
    
    # Base parameters for initialization
    L_INIT = 200
    V_MAX_INIT = 5
    T_GREEN_INIT = 40
    INJECTION_RATE_INIT = 0.1
    P_B_INIT = 0.1
    P_CHG_INIT = 0.8
    P_RED_INIT = 0.1
    P_SKID_INIT = 0.05
    
    # Simulation under NORMAL conditions
    model_normal = IntersectionModel(L_INIT, V_MAX_INIT, T_GREEN_INIT, INJECTION_RATE_INIT, P_B_INIT, P_CHG_INIT, P_RED_INIT, P_SKID_INIT)
    # model_normal.run_simulation(steps=1000)
    # metrics_normal = model_normal.get_metrics()
    
    # Simulation under RAINY conditions
    model_rainy = IntersectionModel(L_INIT, V_MAX_INIT, T_GREEN_INIT, INJECTION_RATE_INIT, P_B_INIT, P_CHG_INIT, P_RED_INIT, P_SKID_INIT)
    # model_rainy.run_simulation(steps=1000, rainy=True)
    # metrics_rainy = model_rainy.get_metrics()
    
    # Print parameter comparison
    print(f"Base Parameters: P_RED={P_RED_INIT}, P_SKID={P_SKID_INIT}, P_CHG={P_CHG_INIT}, P_B={P_B_INIT}")
    
    # Print parameters modified by Rain
    model_rainy.set_weather(Weather.RAINY)
    print(f"\nRAINY Parameters (Modifiers Applied): P_RED={model_rainy.P_RED}, P_SKID={model_rainy.P_SKID}, P_CHG={model_rainy.P_CHG}, P_B={model_rainy.P_B}")
    # print("\nResults (Normal):", metrics_normal)
    # print